In [1]:
!python -V

Python 3.9.7


In [2]:
!pip install pyarrow

In [3]:
!pip install mlflow

# Setup

In [4]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [5]:
!export MLFLOW_TRACKING_URI sqlite:///mlflow.db

/bin/bash: line 0: export: `sqlite:///mlflow.db': not a valid identifier


In [6]:
#from mlflow.tracking import MlflowClient
#MlflowClient
#tracking_uri = MlflowClient.mlflow_tracking_uri

In [7]:
# import mlflow
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment_2")#, artifact_location='.') # mlflow checks, if this experiment exists, if not it is created, if not runs are  appended
#mlflow.set_tracking_uri("nyc-test") #creates local tracking folder
#mlflow.set_experiment("/test")
print("Tracking URI:", mlflow.tracking.get_tracking_uri())

Tracking URI: sqlite:///mlflow.db


# Read the Data

In [8]:
df = pd.read_parquet('../data/green_tripdata_2021-01.parquet')

df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

df = df[(df.duration >= 1) & (df.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

# Data Preparation

In [9]:
train_dicts = df[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

In [10]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [11]:
df_train = read_dataframe('../data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2021-02.parquet')

In [12]:
len(df_train), len(df_val)

(73908, 61921)

In [13]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [14]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [15]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [16]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.75871520559622

In [17]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

# Model - Experiment Tracking with MLflow

In [18]:
with mlflow.start_run(run_name="test_run_1") as run:
    
    mlflow.set_tag("developer", "frauke")
    
    mlflow.log_param("train-data-path", "../data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "../data/green_tripdata_2021-02.parquet")
    
    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

# Second Model - with Hyperparameter Tuning

In [19]:
!pip install hyperopt

In [20]:
!pip install xgboost

In [21]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [22]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [23]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        # log paramters in mlflow
        mlflow.log_params(params)
        booster = xgb.train(
            # paramters are passed to xgboost
            params=params,
            # training on train data
            dtrain=train,
            # set boosting rounds
            num_boost_round=100,
            # validation is done on validation dataset
            evals=[(valid, 'validation')],
            # if model does not improve for 50 methods->stop
            early_stopping_rounds=50
        )
        # make predictions
        y_pred = booster.predict(valid)
        # calculate error
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        # log metric
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [24]:
# define the search space, i.e. the range of parameters for hyperparamter tuning
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0), #[exp(-3), exp(0)] = [0.05, 1]
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

# fmin method tries to minimize the metric
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[15:27:24] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.              
[0]	validation-rmse:12.76520                                                                                                        
[1]	validation-rmse:9.08599                                                                                                         
[2]	validation-rmse:7.64047                                                                                                         
[3]	validation-rmse:7.09458                                                                                                         
[4]	validation-rmse:6.88028                                                                                                         
[5]	validation-rmse:6.78690                                                                                                         
[6]	validation-rmse:6.73811                                          

[60]	validation-rmse:6.65294                                                                                                        
[61]	validation-rmse:6.65375                                                                                                        
[62]	validation-rmse:6.65333                                                                                                        
[63]	validation-rmse:6.65301                                                                                                        
[64]	validation-rmse:6.65299                                                                                                        
[65]	validation-rmse:6.65270                                                                                                        
[66]	validation-rmse:6.65299                                                                                                        
[67]	validation-rmse:6.65265                                         

[20]	validation-rmse:6.52786                                                                                                        
[21]	validation-rmse:6.52325                                                                                                        
[22]	validation-rmse:6.52026                                                                                                        
[23]	validation-rmse:6.51825                                                                                                        
[24]	validation-rmse:6.51618                                                                                                        
[25]	validation-rmse:6.51360                                                                                                        
[26]	validation-rmse:6.51226                                                                                                        
[27]	validation-rmse:6.51021                                         

[81]	validation-rmse:6.47195                                                                                                        
[82]	validation-rmse:6.47178                                                                                                        
[83]	validation-rmse:6.47092                                                                                                        
[84]	validation-rmse:6.46990                                                                                                        
[85]	validation-rmse:6.47142                                                                                                        
[86]	validation-rmse:6.46998                                                                                                        
[87]	validation-rmse:6.46939                                                                                                        
[88]	validation-rmse:6.46905                                         

[41]	validation-rmse:6.65686                                                                                                        
[42]	validation-rmse:6.65605                                                                                                        
[43]	validation-rmse:6.65551                                                                                                        
[44]	validation-rmse:6.65499                                                                                                        
[45]	validation-rmse:6.65404                                                                                                        
[46]	validation-rmse:6.65332                                                                                                        
[47]	validation-rmse:6.65273                                                                                                        
[48]	validation-rmse:6.65213                                         

...

[47]	validation-rmse:6.39645                                                                                                        
[48]	validation-rmse:6.39556                                                                                                        
[49]	validation-rmse:6.39408                                                                                                        
[50]	validation-rmse:6.39264                                                                                                        
[51]	validation-rmse:6.39154                                                                                                        
[52]	validation-rmse:6.39070                                                                                                        
[53]	validation-rmse:6.38939                                                                                                        
[54]	validation-rmse:6.38893                                         

[7]	validation-rmse:7.04299                                                                                                         
[8]	validation-rmse:6.86109                                                                                                         
[9]	validation-rmse:6.73932                                                                                                         
[10]	validation-rmse:6.65640                                                                                                        
[11]	validation-rmse:6.60224                                                                                                        
[12]	validation-rmse:6.55999                                                                                                        
[13]	validation-rmse:6.52840                                                                                                        
[14]	validation-rmse:6.50440                                         

[68]	validation-rmse:6.37553                                                                                                        
[69]	validation-rmse:6.37496                                                                                                        
[70]	validation-rmse:6.37386                                                                                                        
[71]	validation-rmse:6.37370                                                                                                        
[72]	validation-rmse:6.37321                                                                                                        
[73]	validation-rmse:6.37267                                                                                                        
[74]	validation-rmse:6.37288                                                                                                        
[75]	validation-rmse:6.37211                                         

[28]	validation-rmse:6.43689                                                                                                        
[29]	validation-rmse:6.43424                                                                                                        
[30]	validation-rmse:6.43195                                                                                                        
[31]	validation-rmse:6.43003                                                                                                        
[32]	validation-rmse:6.42767                                                                                                        
[33]	validation-rmse:6.42510                                                                                                        
[34]	validation-rmse:6.42331                                                                                                        
[35]	validation-rmse:6.42202                                         

KeyboardInterrupt: 

# Train the Model with a set of selected Parameters

In [ ]:
params =  {'learning_rate': 0.20905792515510074,
            'max_depth': 7,
            'min_child_weight': 0.5241500975917085,
            'objective': 'reg:squarederror',
            'reg_alpha': 0.13309121698466933,
            'reg_lambda': 0.11277257081373988,
            'seed': 42}

In [ ]:
# enable autologging
mlflow.xgboost.autolog()

booster = xgb.train(
            # paramters are passed to xgboost
            params=params,
            # training on train data
            dtrain=train,
            # set boosting rounds
            num_boost_round=1000,
            # validation is done on validation dataset
            evals=[(valid, 'validation')],
            # if model does not improve for 50 methods->stop
            early_stopping_rounds=50
        )

# Model Management

same as above, but save the model as an artifact. Lower the number of boosting rounds to save time.

In [ ]:
with mlflow.start_run(run_name="test_run") as run:
    
    mlflow.set_tag("developer", "frauke")
    
    mlflow.log_param("train-data-path", "../data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "../data/green_tripdata_2021-02.parquet")
    
    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    # save the model as an artifact
    mlflow.log_artifact(local_path="models/lin_reg.bin")#, artifact_path="models_pickle/")

A better way to save the model.

In [ ]:
# disable autolog
mlflow.xgboost.autolog(disable=True)

In [ ]:
with mlflow.start_run():
    best_params =  {'learning_rate': 0.20905792515510074,
                    'max_depth': 7,
                    'min_child_weight': 0.5241500975917085,
                    'objective': 'reg:squarederror',
                    'reg_alpha': 0.13309121698466933,
                    'reg_lambda': 0.11277257081373988,
                    'seed': 42}
    mlflow.log_params(best_params)
        
    booster = xgb.train(
                # paramters are passed to xgboost
                params=params,
                # training on train data
                dtrain=train,
                # set boosting rounds
                num_boost_round=100,
                # validation is done on validation dataset
                evals=[(valid, 'validation')],
                # if model does not improve for 50 methods->stop
                early_stopping_rounds=50
            )
        
    # make predictions
    y_pred = booster.predict(valid)
    # calculate error
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    # log metric
    mlflow.log_metric("rmse", rmse)
     
    # save preprocessor
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    # log preprocessor
    mlflow.log_artifacts("models/preprocessor.b", artifact_path="preprocessor")
    # log the model
    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

# Make Predictions using a saved Model from MLflow

In [58]:
logged_model = 'runs:/dbe24b153d444422bc22eb0baed7a9de/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [59]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: dbe24b153d444422bc22eb0baed7a9de

load as xgboost model

In [62]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

In [63]:
xgboost_model

In [64]:
y_pred = xgboost_model.predict(valid)

In [65]:
y_pred

array([17.384949 ,  7.2070913, 21.283058 , ..., 13.594102 ,  7.2070913,
        8.739125 ], dtype=float32)

# Train some more Models

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

# set autolog
mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):
    print(model_class)
    with mlflow.start_run():
        
        # data path
        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        # log preprocessor
        #mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        
        # log the rmse as metric
        mlflow.log_metric("rmse", rmse)

<class 'sklearn.ensemble._forest.RandomForestRegressor'>
<class 'sklearn.ensemble._gb.GradientBoostingRegressor'>
<class 'sklearn.ensemble._forest.ExtraTreesRegressor'>
